theory

https://jalammar.github.io/illustrated-word2vec/

https://www.notion.so/The-Illustrated-Word2vec-Jay-Alammar-Visualizing-machine-learning-one-concept-at-a-time-5ce312a39291441897a2cd7b333baf33


https://muhark.github.io/python/ml/nlp/2021/10/21/word2vec-from-scratch.html

In [471]:
!pip install torch

In [2]:
import numpy as np
import torch

## Реализуем skip gram

1. Реализация сэмплинга примеров с логикой скользящего окна
2. Формируем батч
    2.1. Собираем примеры позитивных пар итерируясь по тексту.
    2.2  Собираем примеры негативных пар, набирая случаные слова с текста
    Для набранных слов достаём эмбединги из словаря
3. 
3. Как проверять качество?(видимо только ручками)


In [3]:
from torch import nn
import torch

# готовим данные

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [4]:
%%writefile data.txt
Sentence number one sentence sentence sentence.
Sentence number t3wo sentence sentence sentence.
Sentence number 5o sentence sentence sentence.


Overwriting data.txt


In [573]:
from datasets import load_dataset
# tmp_dataset = load_dataset("rotten_tomatoes")
with open('rt.txt', 'w') as f:
    for x in tmp_dataset['train']:
        if len(x['text'].split())<10:
            continue
        f.write(x['text']+'\n')

data loader

In [46]:
from torch.utils.data import Dataset
from typing import Optional, List
from itertools import chain
from collections import Counter
import re
import string




def _preprocess_message_to_tokens(text: str):
    sub_pattern = f'[{re.escape(string.punctuation)}]'
    tmp_str = re.sub(sub_pattern, ' ', text.lower())
    return tmp_str.split()


class CustomDataset(Dataset):
    def __init__(self, text_fpath: str, max_seq_len: int):
        self.max_seq_len = max_seq_len
        with open(text_fpath, 'r') as f:
            self.lines = f.readlines()
        
    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        # load text with id = idx        
        return self.preproc(self.lines[idx])
    
    def preproc(self, x: str):
        tokens = _preprocess_message_to_tokens(x.lower())
        padded_tokens = CustomDataset.do_padding(tokens, self.max_seq_len)
        return padded_tokens
    
    @staticmethod
    def do_padding(tokens: List[str], max_seq_len: int) -> List[str]:
        tokens = tokens[:max_seq_len]
        padded_tail = ['PAD_TOKEN']*(max_seq_len-len(tokens))
        return tokens + padded_tail



def prepare_vocab(dataset: Dataset, max_size: Optional[int] = None):
    vocab = Counter(chain.from_iterable(map(dataset.preproc, dataset.lines)))
    words_lst = map(lambda x: x[0], vocab.most_common(max_size))
    return dict((v, k) for k, v in enumerate(words_lst))


MAX_SEQ_LEN = 12
texts_dataset = CustomDataset('rt.txt', MAX_SEQ_LEN)
# texts_dataset = CustomDataset('data.txt', MAX_SEQ_LEN)
vocab = prepare_vocab(texts_dataset, )

In [172]:
from torch.utils.data import DataLoader

import numpy as np
from typing import List
from itertools import chain, repeat
import random

# import os


def construct_negative_pairs(num_texts, max_seq_len, window_size, count):
    negative_texts_pairs = np.random.choice(np.arange(num_texts), size=int(count*2), replace=True)\
                                        .reshape(-1, 2)
    words_idxs = np.random.randint(max_seq_len, size=(negative_texts_pairs.shape[0], window_size, 2))
    yield from zip(negative_texts_pairs, words_idxs)



def pairs_sentence(sentence: List[str], window_size: int):
    half_win = (window_size-1)//2
    for center_idx in range(half_win, len(sentence)-half_win, 1):
        start, end = center_idx-half_win, center_idx+half_win
        for context_idx in range(window_size):
            if context_idx==half_win:
                continue
            yield vocab[sentence[center_idx]], vocab[sentence[context_idx+start]]

def construct_positive_pairs(splits: List[List[str]], window_size: int):        
    return chain.from_iterable(list(map(lambda x: pairs_sentence(x, window_size), splits)))
    
    
def collate_fn(batch):
    len_idxs = len(batch)
    splits = batch
    
    batch_X = []
    batch_y = []

    for t_idxs, words_idxs in construct_negative_pairs(len(splits), 
                                                       MAX_SEQ_LEN, WINDOW_SIZE, 
                                                       count=len(splits)):
        text1, text2 = splits[t_idxs[0]], splits[t_idxs[1]]
        for w1_idx, w2_idx in words_idxs:
            batch_X.append((vocab[text1[w1_idx]], vocab[text2[w2_idx]]))
            batch_y.append(0)
    batch_X, batch_y  = batch_X[:MAX_BATCH_SIZE], batch_y[:MAX_BATCH_SIZE]
    pos_pairs = list(construct_positive_pairs(splits, WINDOW_SIZE))
    
    n = len(pos_pairs)
    m = MAX_BATCH_SIZE
    for pair in pos_pairs:
        if not (random.random()>(1-(m/n))):
            continue
        batch_X.append(pair)
        batch_y.append(1)
    return torch.tensor(batch_X), torch.tensor(batch_y, dtype=torch.float32)



WINDOW_SIZE = 7
MAX_BATCH_SIZE = 50
dataloader = DataLoader(texts_dataset, batch_size=5, shuffle=True, collate_fn=collate_fn)
X, y = next(iter(dataloader))
len(X), len(y), torch.mean(y)

(84, 84, tensor(0.5833))

model

In [173]:

# def get_batch_pair_scores(embeddings: nn.Embedding, 
#                    batch_idxs: torch.Tensor) -> torch.Tensor:
#     embs = embedding(batch_idxs)
#     input_words = embs[:, 0, :]
#     target_words = embs[:, 1, :]
#     mult = torch.multiply(input_words,  target_words)
#     return  torch.sum(mult, axis=-1)

class TinyModel(torch.nn.Module):
    def __init__(self, vocab, dim, padding_idx):
        super(TinyModel, self).__init__()
        self.embedding = nn.Embedding(len(vocab), dim, padding_idx=padding_idx)
        self.linear = torch.nn.Linear(dim, dim)
        
    def forward(self, x):
        x = self.embedding(x)
        input_words = x[:, 0, :]
        target_words = x[:, 1, :]
        input_words, target_words = self.linear(input_words), self.linear(target_words)
        mult = torch.multiply(input_words,  target_words)
        return  torch.sum(mult, axis=-1)



In [175]:
from tqdm.auto import tqdm
import torch.optim as optim

def train(model, dataloader, epochs: int, lr: int=1e-4):
    loss_fn = torch.nn.BCELoss()
    # tinymodel.forward(torch.tensor([[[0, 1], [1,2]]]))
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    losses = []
    try:    
        for epoch in range(epochs):
            losses = []
            print(f'Epoch: {epoch+1:3d}; ', end='')
            pbar = tqdm(dataloader)
            for i, (X, y) in enumerate(pbar):
                optimizer.zero_grad()
                pair_output = model(X)
                output = torch.sigmoid(pair_output)
                loss = loss_fn(output, y)
                loss.backward()
                optimizer.step()
                loss_score = loss.detach().numpy()
                losses.append(loss_score)
                if i % 100 == 0:
                    pbar.set_description(f'{np.mean(losses):.4f}')
    finally:
        return losses
dim = 50
tinymodel = TinyModel(vocab, dim, padding_idx=vocab['PAD_TOKEN'])
train(tinymodel, dataloader, epochs=1000, lr=5*1e-4);

Epoch:   1; 

  0%|          | 0/1507 [00:00<?, ?it/s]

Epoch:   2; 

  0%|          | 0/1507 [00:00<?, ?it/s]

Epoch:   3; 

...

  0%|          | 0/1507 [00:00<?, ?it/s]

Epoch: 247; 

  0%|          | 0/1507 [00:00<?, ?it/s]

Epoch: 248; 

  0%|          | 0/1507 [00:00<?, ?it/s]

In [178]:
vocab_words = list(vocab.keys())
tmp = tinymodel.embedding(torch.tensor([vocab[w] for w in vocab_words])).detach().numpy()
words_embs = dict(zip(vocab_words, tmp))
words_embs;

In [197]:
from scipy import spatial

result = 1-spatial.distance.cosine(words_embs['movie'], words_embs['dog'])
result

0.13034085929393768

In [202]:
import joblib

joblib.dump
joblib.dump({
    'embs': words_embs,
    'epochs': 248,
    'vocab': vocab,
    'lr': 5*1e-4
}, 'trained_vec.j')

['trained_vec.j']

In [205]:
!ls -lah

total 4,8M
drwxrwxr-x  3 grigory grigory  4,0K дек 30 10:51 .
drwxrwxr-x 49 grigory grigory  4,0K дек 29 19:10 ..
-rw-rw-r--  1 grigory grigory   144 дек 30 09:19 data.txt
drwxrwxr-x  2 grigory grigory  4,0K дек 29 19:10 .ipynb_checkpoints
-rw-rw-r--  1 grigory grigory  921K дек 29 22:01 rt.txt
-rw-rw-r--  1 grigory grigory  2,9M дек 30 10:51 trained_vec.j
-rw-rw-r--  1 grigory grigory 1017K дек 30 10:51 word2vec_scratch.ipynb
